In [11]:
import pandas as pd
import numpy as np
import tensorflow as tf

from keras.layers import Input, Embedding, Dense, SpatialDropout1D, Bidirectional, LSTM
from keras.models import Model
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout

from keras import backend as K
from keras.layers import Layer
from keras import initializers, regularizers, constraints, Input

In [2]:
# load numpy array from csv file
from numpy import loadtxt
# load array
X_train = loadtxt('x_train_small.csv', delimiter=',')
X_train_hyp = loadtxt('x_train_hyp_small.csv', delimiter=',')
Y_train = loadtxt('y_train_small.csv', delimiter=',')
# print the array
X_train

array([[2., 6., 9., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       ...,
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.]])

In [3]:
X_train_hyp

array([[2., 6., 9., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       ...,
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.],
       [2., 6., 6., ..., 0., 0., 0.]])

In [4]:
Y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
# only for testing
X_train_hyp = X_train_hyp[:, :100]

In [6]:
VOCAB_SIZE = 1254
INPUT_LENGTH = 100 #3000
EMBEDDING_DIM = 128

In [7]:
# custom dot product function
def dot_product(x, kernel):
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [8]:
# find a way to return attention weight vector a
class AttentionWithContext(Layer):
    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        # initialization of all learnable params
        self.init = initializers.get('glorot_uniform')
        
        # regularizers for params, init as None
        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
        
        # constraints for params, init as None
        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
#         assert len(input_shape) == 3
        
        # weight matrix
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        # bias term
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        
        # context vector
        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)
        
    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)
        
        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
#         a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [12]:
# model
def build_model(vocab_size, embedding_dim, input_length):
    sequence_input = Input(shape=(input_length,), dtype='int32')
    embedded_sequences = Embedding(vocab_size, embedding_dim, input_length=input_length)(sequence_input)
    output_1 = SpatialDropout1D(0.2)(embedded_sequences)
    output_2 = Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))(output_1)
    context_vec = AttentionWithContext()(output_2)
    predictions = Dense(41, activation='softmax')(context_vec)
    model = Model(inputs=sequence_input, outputs=predictions)
    return model

In [13]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, INPUT_LENGTH)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 128)          160512    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 100, 128)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 256)          263168    
_________________________________________________________________
attention_with_context_1 (At (None, 256)               66048     
_________________________________________________________________
dense_1 (Dense)              (None, 41)                10537     
Total params: 500,265
T

In [18]:
epochs = 10
batch_size = 64

history = model.fit(X_train_hyp, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1)
# callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 180 samples, validate on 20 samples
Epoch 1/10
180/180 [==============================] - 2s 11ms/step - loss: 2.7257 - acc: 0.1444 - val_loss: 2.9155 - val_acc: 0.1000
Epoch 2/10
180/180 [==============================] - 2s 11ms/step - loss: 2.7210 - acc: 0.1722 - val_loss: 2.8962 - val_acc: 0.1000
Epoch 3/10
180/180 [==============================] - 2s 10ms/step - loss: 2.7223 - acc: 0.1278 - val_loss: 2.8928 - val_acc: 0.0500
Epoch 4/10
180/180 [==============================] - 2s 11ms/step - loss: 2.7184 - acc: 0.1889 - val_loss: 2.9036 - val_acc: 0.1000
Epoch 5/10
180/180 [==============================] - 2s 11ms/step - loss: 2.7186 - acc: 0.1667 - val_loss: 2.8853 - val_acc: 0.1000
Epoch 6/10
180/180 [==============================] - 2s 11ms/step - loss: 2.6989 - acc: 0.1611 - val_loss: 2.8811 - val_acc: 0.0500
Epoch 7/10
180/180 [==============================] - 2s 11ms/step - loss: 2.6959 - acc: 0.1944 - val_loss: 2.8789 - val_acc: 0.0000e+00
Epoch 8/10
180/180 [

In [50]:
example_x = X_train_hyp[0]
print(np.shape(example_x))
temp = model.predict(X_train_hyp)
# print(len(temp)), temp
print(np.sum(temp[0]))
print(temp[0])
for i in temp:
    print(np.argmax(i))


(100,)
1.0
[0.0243152  0.02401842 0.02335423 0.02598605 0.02421737 0.02615896
 0.02448767 0.02405805 0.02501558 0.02388974 0.02425782 0.02398404
 0.0241027  0.02509943 0.02363893 0.02460951 0.02450567 0.02408865
 0.02421672 0.02386107 0.02431127 0.02435374 0.02531658 0.02412735
 0.02531849 0.02429923 0.02481478 0.02422279 0.0248631  0.02427559
 0.02428459 0.02464528 0.02424267 0.02359925 0.02461487 0.02359962
 0.02410334 0.02545464 0.02404701 0.02386709 0.02377296]
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
3
5
5
5
5
5
5
5
5
5
3
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
